# Export top of the book & trades using dedicated service - C#

### Overview
Retrieving trades and top of the book data points over a look back period is a valuable indicator to highlight market activity. It can be correlated with a set of technical indicators to better analyze market movements and build a decision making parameter.

This sample is designed to request simultaneously trades and top of the book and illustrate market activity over a look back period.

<div class="alert alert-block alert-info">
    <b>Note:</b> To highlight individual tradres among market activity. Please refer to <i><a href="">bestex-pointintime</a></i> and <i><a href="">bestex-interval</a></i> notebooks.
</div>

This sample demonstrates how to request and plot from a dedicated data service **on-demand** trades over the time.

### Inputs/outputs
Trades and best Bid/Ask extraction sample requires instrument's identifier, date time intervals as per inputs and returns top fo the book the occured trades data points.

### Services used
This sample uses *gRPC requests* in order to retrieve trades and top of the book from the hosted service. The queried endpoint in this script are:
* *TickTradesAndBookService*: to directly retrieve trades and top of the book data from the server.

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.Tick.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    
***

# Run Top of the book & Trades extraction sample

### Step 1: Install packages

In [ ]:
#i "nuget: file:///home/jovyan/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis"
#r "nuget:CsvHelper"

In [ ]:
using Systemathics.Apis.Type.Shared.V1;
using Systemathics.Apis.Services.Tick.V1;
using Google.Protobuf.WellKnownTypes;
using Google.Type;
using Grpc.Net.Client;
using Grpc.Core;
// csv extraction
using CsvHelper; 
using CsvHelper.Configuration;
using System.IO;
using System.Globalization;

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Retrieve authentication token
var token = System.Environment.GetEnvironmentVariable("AUTH0_TOKEN");

// Add token to the request header
var headers = new Metadata();
headers.Add("Authorization", $"Bearer {token}");

// Define API entry
var channel = GrpcChannel.ForAddress("https://apis.systemathics.cloud");

In [ ]:
// Display authentication token 
display(token);

### Step 3: Create and process request
To request *tick trades and book* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Tick data request parameters

#### 3.1 Instrument selection

In [ ]:
var ticker = "AAPL";
var exchange = "BATS";

#### 3.2 Time period delimitation

In [ ]:
// Create time intervals
var start = new DateTime(2021, 03, 05);
var end = new DateTime(2021, 03, 05);
var dateIntervals = new DateInterval()
{
    StartDate = new Date { Year = start.Year, Month = start.Month, Day = start.Day },
    EndDate = new Date { Year = end.Year, Month = end.Month, Day = end.Day }
};

// Build the market data request time interval (we are using Google date time format : we have to cast the dates)
// UTC time zone
var timeInterval = new TimeInterval()
{
    StartTime = new TimeOfDay { Hours = 14, Minutes = 00, Seconds = 00 },
    EndTime = new TimeOfDay { Hours = 15, Minutes = 00, Seconds = 00 }    
};

#### 3.3 Request creation
The following code snippet creates *gRPC client*, process request and returns the request reply:

In [ ]:
// Generate constraints based on the previous time selection:
var constraints = new Constraints(); 
constraints.DateIntervals.Add(dateIntervals);
constraints.TimeIntervals.Add(timeInterval);

The following code snippet create the market data request and instantiate the service:

In [ ]:
// Generate the tick data request
var request = new TickTradesAndBookRequest { Constraints = constraints };
var identifier = new Identifier { Exchange = exchange, Ticker = ticker };
request.Identifiers.Add(new [] {identifier}); 

// Instantiate the tick data service
var service = new TickTradesAndBookService.TickTradesAndBookServiceClient(channel);

### Step 4: Export data

#### 4.1 Retrieve top of the book & trades

The following code snippet set the header fields:

The following code snippet stores data points for *trades*, *best bid* and *best ask*:

In [ ]:
// Process tick data stream service: get price / size / timestamp
var data = new List<TradeAndBook>();
var call = service.TickTradesAndBook(request, headers);
await foreach (var current in call.ResponseStream.ReadAllAsync())
{
    // Store data only (skip the mapping data)
    if (current.Data != null) 
    {
       data.Add(current.Data);
    }   
}
display(data);

The following code snippet displays request results count:

In [ ]:
// Display trade and top the book data points
Console.WriteLine($"Total extracted data: {data.Count:N0} data points");

#### 4.2 Export top of the book & trades

The following code snippets export results to *csv file*:

In [ ]:
public sealed class DataMap : ClassMap<TradeAndBook>
{
    public DataMap()
    {
        var i = 0;
        
        Map(m => m.TimeStamp).Name("Timestamp").Index(i++);
    
        Map(m => (m.Book != null) ? m.Book.Bid.Price : 0.0).Name("Bid Price").Index(i++);
        Map(m => (m.Book != null) ? m.Book.Bid.Size : 0).Name("Bid Size").Index(i++);

        Map(m => (m.Book != null) ? m.Book.Ask.Price : 0.0).Name("Ask Price").Index(i++);
        Map(m => (m.Book != null) ? m.Book.Ask.Size : 0).Name("Ask Size").Index(i++);
        
        Map(m => (m.Trade != null) ? m.Trade.Price : 0.0).Name("Trade Price").Index(i++);
        Map(m => (m.Trade != null) ? m.Trade.Size : 0).Name("Trade Size").Index(i++);
         
    }
}

In [ ]:
public sealed class DataMap : ClassMap<TradeAndBook>
{
    public DataMap()
    {
        var i = 0;
        
        Map(m => m.TimeStamp).Name("Timestamp").Index(i++);
    
        Map(m => m.Book.Bid.Price)?.Name("Bid Price").Index(i++);
        Map(m => m.Book.Bid.Size)?.Name("Bid Size").Index(i++);

        Map(m => m.Book.Ask.Price)?.Name("Ask Price").Index(i++);
        Map(m => m.Book.Ask.Size)?.Name("Ask Size").Index(i++);
        
        Map(m => m.Trade.Price)?.Name("Trade Price").Index(i++);
        Map(m => m.Trade.Size)?.Name("Trade Size").Index(i++); 
    }
}

In [ ]:
using (var writer = new StreamWriter($"L1L2_{ticker}.csv"))
using (var csv = new CsvWriter(writer, CultureInfo.InvariantCulture))
{
    csv.Context.RegisterClassMap<DataMap>();
    csv.WriteRecords(data);
}